<a href="https://colab.research.google.com/github/amirpaia/blenderbot/blob/main/blender_finetuning_with_reddit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.Installing prerequisites

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Jun  7 17:03:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
mydrive_path = '/content/drive/MyDrive/colabs/'

Mounted at /content/drive


In [ ]:
# !pip uninstall -q parlai
!pip install -q parlai
!pip install transformers

     |████████████████████████████████| 1.7 MB 33.3 MB/s 
     |████████████████████████████████| 175 kB 73.3 MB/s 
     |████████████████████████████████| 138 kB 77.5 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
     |████████████████████████████████| 248 kB 83.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 76 kB 5.4 MB/s 
     |████████████████████████████████| 64 kB 3.0 MB/s 
     |████████████████████████████████| 46 kB 3.7 MB/s 
     |████████████████████████████████| 132 kB 82.5 MB/s 
     |████████████████████████████████| 2.7 MB 69.3 MB/s 
     |████████████████████████████████| 40 kB 5.6 MB/s 
     |████████████████████████████████| 208 kB 73.4 MB/s 
     |████████████████████████████████| 346 kB 79.1 MB/s 
     |████████████████████████████████| 235 kB 86.9 MB/s 
     |███████████████████████

# 1.Preparing Data

## Genreal Functions

In [ ]:
def transfer_list_of_turns_to_dialog(d):
    if len(d)%2 !=0: d = d[:-1]
    t = ""
    for i in range(0,len(d),2):
        u1 = d[i]
        u2 = d[i+1]

        if (i+2) != len(d):
            t += "text:"+u1+"\t"+"labels:"+u2+"\n"
        else:
            t += "text:"+u1+"\t"+"labels:"+u2+"\t"+"episode_done:True"+"\n"
    return t

def transfer_list_of_pairs_to_dialog(d):
  t = ""
  for i, text_label_pair in enumerate(d):
    u1 = text_label_pair[0]
    u2 = text_label_pair[1]

    if i != (len(d) - 1):
      t += "text:"+u1+"\t"+"labels:"+u2+"\n"
    else:
      t += "text:"+u1+"\t"+"labels:"+u2+"\t"+"episode_done:True"+"\n"

  return t

def convert_parlai_format_to_list_of_turns(lines):
    result = []
    for line in lines:
        text_label = line.split("\t")
        result.append(text_label[0].replace("text:", ""))
        result.append(text_label[1].replace("labels:", "").replace("\n",""))
    return result

t = ['hello','how are you','good','bye','test']
print(transfer_list_of_turns_to_dialog(t))

t = [['hello','how are you'],['good','bye']]
print(transfer_list_of_pairs_to_dialog(t))

text:hello	labels:how are you
text:good	labels:bye	episode_done:True

text:hello	labels:how are you
text:good	labels:bye	episode_done:True



## French Reddit

In [ ]:
data_path = f"{mydrive_path}aliae-workspace/datasets/french_reddit/"
data_path

'/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_reddit/'

In [ ]:
with open(f"{data_path}/data_train.txt") as f:
    lines = f.readlines()
lines[:10]

["text:Pour ceux qui jouent à LoL et qui attendaient impatiemment le URF, ça sera au final le NURF qui sera là. L'inverse du Urf : cooldown augmentés, conso mana/énergie augmentée, dégâts des minions augmentés... Voilà, enjoy, moi je vais pleurer dans un coin parce que le urf était le mode qui me poussait à garder LoL installé. ^^^^quelqu'un ^^^^n'aime ^^^^pas ^^^^LoL ^^^^? ^^^^:)\tlabels:Ça, ou alors demain c'est le 1^er avril.\n",
 "text:Le vrai mode URF était déjà une blague du 1er avril. Sauf que ça a bien marché et ils ont laissé le mode 2 semaines.On verra bien demain de toute façon :(\tlabels:Et tu penses vraiment qu'ils vont remplacer un mode de jeu que tout le monde attend depuis 1 an par un truc pas fun du tout ?C'est clairement une blague.\n",
 "text:Riot est tellement une vaste blague comme entreprise qu'ils en seraient capable D:\tlabels:Une vaste blague comme entreprise ? Les mecs ont réussi, en pompant le concept d'un jeu existant à créer le jeu en ligne le plus joué au 

# 2.Finetuning

In [ ]:
finetuned_model_path = f'{mydrive_path}blender-models/finetuned-reddit-400m/'
init_model = 'zoo:blender/blender_400Mdistill/model'
dict_file  = 'zoo:blender/blender_400Mdistill/model.dict'
# init_model = 'zoo:blender/blender_90M/model'
# dict_file  = 'zoo:blender/blender_90M/model.dict'
finetuned_model_path

'/content/drive/MyDrive/colabs/blender-models/finetuned-reddit-400m/'

In [ ]:
# 90M settings
# !rm -rf $finetuned_model_path
!mkdir -p $finetuned_model_path


from parlai.scripts.train_model import TrainModel

TrainModel.main(
    # task
    # task= "french_blended_skill_talk,french_xpersona,french_empathetic_dialogues",
    # multitask_weights= "1,3,3",

    task='fromfile:parlaiformat', 
    fromfile_datapath= f'{data_path}data',
    fromfile_datatype_extension=True,

    model='transformer/generator',
    model_file= f'{finetuned_model_path}model',
    
    # initialize with a pretrained model
    init_model= init_model,
    dict_file=dict_file,
    
    # arguments we get from the pretrained model.
    # Unfortunately, these must be looked up separately for each model.
    n_heads=16, n_layers=8, n_positions=512, text_truncate=512,
    label_truncate=128, ffn_size=2048, embedding_size=512,
    activation='gelu', variant='xlm',
    dict_lower=True, dict_tokenizer='bpe',
    
    # depend on your gpu. 
    validation_every_n_epochs=0.25,
    num_epochs = 5,
    log_every_n_secs= 60,
    verbose = True,
    batchsize= 8, 
    fp16= True, fp16_impl= "mem_efficient",
    
    # arguments we get from the pretrained model.
    
    # speeds up validation
    skip_generation=True,
    vp= 10,
    validation_metric= "ppl", #vmt = "ppl"
    validation_metric_mode= "min", # vmm= "min"
    
    # helps us cram more examples into our gpu at a time
    dynamic_batching='full',

    
    # some training arguments, specific to this fine-tuning
    lr=1e-5, optimizer='adam',
    attention_dropout= 0.0, 
    model_parallel= False,
    warmup_updates=100,

    # customized parameters
    # inference= "beam"
    # beam_min_length= 20,
    # beam_block_ngram= 3,
    # beam_context_block_ngram= 3,
    # beam_size= 10,
)

In [ ]:
# mydrive_path = '/content/finetuned-multitask-400m-double-sided-2epochs'
# mydrive_path = '/content/drive/MyDrive/colabs/blender-models/finetuned-multitask-400m-double-sided/'

In [ ]:
# 400M settings
# !rm -rf $finetuned_model_path
!mkdir -p $finetuned_model_path


from parlai.scripts.train_model import TrainModel

TrainModel.main(
    # task
    # task= "french_blended_skill_talk,french_xpersona,french_empathetic_dialogues",
    # multitask_weights= "1,3,3",

    task='fromfile:parlaiformat', 
    fromfile_datapath= f'{data_path}data',
    fromfile_datatype_extension=True,

    model='transformer/generator',
    model_file= f'{finetuned_model_path}/model',
    
    # initialize with a pretrained model
    init_model= init_model,
    dict_file=dict_file,
    
    # depend on your gpu
    validation_every_n_epochs=0.25, # veps= 0.25, 
    num_epochs = 0.25,
    log_every_n_secs= 300,
    verbose = True,
    attention_dropout= 0.0, 
    batchsize= 16, 
    fp16= True, fp16_impl= "mem_efficient",
    # save_after_valid= True,

    # arguments we get from the pretrained model. "from recipes page for 2.7B model" 
    embedding_size= 1280, ffn_size= 5120,
    variant= "prelayernorm",
    n_heads= 32, n_positions= 128, 
    n_encoder_layers= 2, n_decoder_layers= 12,

    label_truncate= 128, text_truncate= 128, truncate= 128,
    activation= "gelu",
    history_add_global_end_token= "end", 
    delimiter= '  ', 
    dict_tokenizer= "bytelevelbpe",
    dropout= 0.1,
    
    # some training arguments, specific to this fine-tuning
    lr= 7e-06, lr_scheduler= "reduceonplateau", lr_scheduler_patience= 3,
    optimizer= "mem_eff_adam",
    relu_dropout= 0.0, 
    model_parallel= False,
    warmup_updates= 100,
    update_freq= 2,
    gradient_clip= 0.1, 

    # speeds up validation
    skip_generation= True,
    vp= 10,
    validation_metric= "ppl", #vmt = "ppl"
    validation_metric_mode= "min", # vmm= "min"

    # customized parameters
    # inference = 'topk', 
    # temperature = 0.7, 
    # topk=30, 
    # beam_length_penalty=1.03
)

17:30:26 | building dictionary first...
17:30:26 | No model with opt yet at: /content/drive/MyDrive/colabs/blender-models/finetuned-reddit-400m//model(.opt)
17:30:26 | your model is being loaded with opts that do not exist in the model you are initializing the weights with: download_path: None,verbose: True,is_debug: False,datapath: /usr/local/lib/python3.7/dist-packages/data,final_extra_opt: ,eval_dynamic_batching: None,num_workers: 0,max_train_steps: -1,log_every_n_steps: 50,validation_every_n_steps: -1,load_from_checkpoint: True,world_logs: ,save_format: conversations,log_keep_fields: all,wandb_log: False,wandb_name: None,wandb_project: None,wandb_entity: None,mutators: None,fromfile_datapath: /content/drive/MyDrive/colabs/aliae-workspace/datasets/french_reddit/data,fromfile_datatype_extension: True,checkpoint_activations: False,interactive_mode: False
17:30:26 | your model is being loaded with opts that differ from the model you are initializing the weights with. Add the following 

({'clen': AverageMetric(324.8),
  'ctpb': GlobalAverageMetric(1829),
  'ctps': GlobalTimerMetric(1.423e+04),
  'ctrunc': AverageMetric(0.7601),
  'ctrunclen': AverageMetric(210.3),
  'exps': GlobalTimerMetric(124.3),
  'exs': SumMetric(4.19e+04),
  'gpu_mem': GlobalAverageMetric(0.2818),
  'llen': AverageMetric(80.77),
  'loss': AverageMetric(3.658),
  'lr': GlobalAverageMetric(6.93e-06),
  'ltpb': GlobalAverageMetric(1062),
  'ltps': GlobalTimerMetric(8260),
  'ltrunc': AverageMetric(0.1888),
  'ltrunclen': AverageMetric(14.33),
  'ppl': PPLMetric(38.79),
  'token_acc': AverageMetric(0.3014),
  'token_em': AverageMetric(9.547e-05),
  'total_train_updates': GlobalFixedMetric(2618),
  'tpb': GlobalAverageMetric(2890),
  'tps': GlobalTimerMetric(2.249e+04)},
 {'clen': AverageMetric(324.8),
  'ctpb': GlobalAverageMetric(1828),
  'ctps': GlobalTimerMetric(1.43e+04),
  'ctrunc': AverageMetric(0.7584),
  'ctrunclen': AverageMetric(210.5),
  'exps': GlobalTimerMetric(125),
  'exs': SumMetric(

In [ ]:
# !cp -rv /content/finetuned-multitask-400m-double-sided-2epochs/* /content/drive/MyDrive/colabs/blender-models/finetuned-multitask-400m-double-sided/
# !ls -lah /content/finetuned-multitask-400m-double-sided-2epochs/
# !ls -lah /content/finetuned-multitask-400m-double-sided/

# 4.Display Model

In [ ]:
from parlai.scripts.display_model import DisplayModel
DisplayModel.main(
    # task='french_blended_skill_talk',
    task='fromfile:parlaiformat', 
    fromfile_datapath= f'{data_path}data',
    fromfile_datatype_extension=True,
    datatype= "test",

    model_file= f'{finetuned_model_path}/model',
    num_examples=20,
    skip_generation=False,

    # the result of grid search on 400M model and BST dataset when inference=topk
    beam_block_ngram= 2,
	beam_context_block_ngram= 3,
	beam_length_penalty= 1,
	beam_min_length= 10,
	beam_size= 20,
	inference= "topk",
	temperature= 0.5,
	topk= 20,
	topp= 0.9

    # # Farnaz sent me
    # beam_block_ngram= 3,
    # beam_context_block_ngram= 3,
    # beam_min_length= 20, 
    # beam_size= 10,
    # inference =  'topk',  
    # topk=20, 
    # temperature = 0.5, 
    # beam_length_penalty=0.8
)

15:05:29 | Overriding opt["datatype"] to test (previously: train)
15:05:29 | Overriding opt["model_file"] to /content/drive/MyDrive/colabs/blender-models/finetuned-reddit-90m//model (previously: /content/drive/MyDrive/colabs/blender-models/finetuned-reddit-90m/model)
15:05:29 | Overriding opt["skip_generation"] to False (previously: True)
15:05:29 | Overriding opt["beam_block_ngram"] to 2 (previously: -1)
15:05:29 | Overriding opt["beam_context_block_ngram"] to 3 (previously: -1)
15:05:29 | Overriding opt["beam_length_penalty"] to 1.0 (previously: 0.65)
15:05:29 | Overriding opt["beam_min_length"] to 10 (previously: 1)
15:05:29 | Overriding opt["beam_size"] to 20 (previously: 1)
15:05:29 | Overriding opt["inference"] to topk (previously: greedy)
15:05:29 | Overriding opt["temperature"] to 0.5 (previously: 1.0)
15:05:29 | Overriding opt["topk"] to 20 (previously: 10)
15:05:29 | Using CUDA
15:05:29 | loading dictionary from /content/drive/MyDrive/colabs/blender-models/finetuned-reddit-90

In [ ]:
# from parlai.scripts.display_data import DisplayData
# DisplayData.main(task='empathetic_dialogues', num_examples=10)

from parlai.scripts.display_data import DisplayData
DisplayData.main(
    task='fromfile:parlaiformat', 
    fromfile_datapath= f'{data_path}data',
    fromfile_datatype_extension=True,
    # model_file= f'{finetuned_model_path}/model',
    # dict_file= dict_file,

    num_examples=20,
    # skip_generation=False,

    # beam_min_length= 20,
    # beam_block_ngram= 3,
    # beam_context_block_ngram= 3,
    # beam_size= 10,

    # inference= "beam"
)

09:07:59 | Opt:
09:07:59 |     allow_missing_init_opts: False
09:07:59 |     batchsize: 1
09:07:59 |     datapath: /usr/local/lib/python3.7/dist-packages/data
09:07:59 |     datatype: train:ordered
09:07:59 |     dict_class: None
09:07:59 |     display_add_fields: 
09:07:59 |     download_path: None
09:07:59 |     dynamic_batching: None
09:07:59 |     fromfile_datapath: /content/drive/MyDrive/colabs/aliae-workspace/datasets/french_reddit/data
09:07:59 |     fromfile_datatype_extension: True
09:07:59 |     hide_labels: False
09:07:59 |     ignore_agent_reply: True
09:07:59 |     image_cropsize: 224
09:07:59 |     image_mode: raw
09:07:59 |     image_size: 256
09:07:59 |     init_model: None
09:07:59 |     init_opt: None
09:07:59 |     is_debug: False
09:07:59 |     loglevel: info
09:07:59 |     max_display_len: 1000
09:07:59 |     model: None
09:07:59 |     model_file: None
09:07:59 |     multitask_weights: [1]
09:07:59 |     mutators: None
09:07:59 |     num_examples: 20
09:07:59 |    